# Install tensorflow if not already done

# Import all the required modules

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.layers import Dropout, Input
from sklearn.metrics import accuracy_score

# Read the csv file

In [ ]:
df = pd.read_csv("animals.csv")

In [ ]:
print(df.head(1))

# Drop the "winner" column and save it as y

In [ ]:
x = df.drop(columns= 'winner', axis = 1)   # Drop the "winner" column
x = x.dropna(axis=1, how ='all')           # Drop the column with all NA
y = df['winner']                           # Save the column as y

In [ ]:
x.shape

# Data standardisation

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(x)

In [ ]:
standard_data = scaler.transform(x)
print(standard_data)

In [ ]:
x = standard_data

# Train, Test, Split

In [ ]:
# test size means fraction of test
# stratify ensures that for y i.e the outcome both 0 and 1 occur in equal proportion in both test and train
# random state to ensure reproducibilty

x_train, x_test, y_train, y_test = train_test_split( x, y, test_size = 0.2, stratify = y, random_state = 42)

In [ ]:
print(x.shape, x_train.shape, x_test.shape)

# Training the model using neural network

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)  # For numpy-related random processes
tf.random.set_seed(42)  # For TensorFlow-related random processes


model = Sequential()
model.add(Input(shape=(x_train.shape[1],)))
model.add(Dropout(0.5))  # Dropout layer for uncertainty
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Dropout layer for uncertainty
model.add(Dense(2, activation='softmax'))

model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=1)

# Function to predict with Monte Carlo Dropout
def mc_predict(model, x_test, num_samples=100):
    preds = [model(x_test, training=True) for _ in range(num_samples)]
    return np.mean(preds, axis=0), np.std(preds, axis=0)  # Mean and standard deviation

# Predict probabilities and uncertainty
y_prob_mean, y_prob_std = mc_predict(model, x_test)

print("Predicted Probabilities:", y_prob_mean)
print("Uncertainty (Std Dev):", y_prob_std)


# Evaluation of model

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=1)

# Print the accuracy
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Making a predictive system

In [ ]:
def animalvs(input_data):


    # changing the input data to numpy array

    input_data_as_numpy = np.asarray(input_data)

    # reshape the array as we are predicting for one instance

    input_data_reshaped = input_data_as_numpy.reshape(1,-1)

    # standardize the array

    std_data = scaler.transform(input_data_reshaped)
    prediction = model.predict(std_data)
    print(prediction[0])

    if (prediction[0][0]==0.5):
      print(" it is a tie")
    elif(prediction[0][0]>0.5):
      print(f" there is  a {prediction[0][0]*100:.2f} % chance that the first animal will win ")
    else:
      print(f" there is  a {prediction[0][1] *100:.2f} % chance that the second animal will win ")




# Evaluation with data

In [ ]:
# walrus vs nile crocodile : the winner is nile crocodile

input_data = (1200, 4.5, 35, 6.5, 200, 40, 8, 9, 7,	500, 4.5, 35, 9.5, 4400, 50, 9, 7.5, 8)
animalvs(input_data)

In [ ]:
# polar bear vs nile crocodile : the winner is polar bear

input_data =(550,5.5,40,8.5,1200,50,8,9.5,9.5,500,4.5,35,9.5,4400,50,9,7.5,8)
animalvs(input_data)

In [ ]:
# african lion vs bengal tiger : the winner is bengal tiger

input_data =(190,6.5,58,9,650,12,6,4.5,7.5,230,6,65,8.5,1050,13,7,4.5,8)
animalvs(input_data)

In [ ]:
# rhino vs hippo : the winner is rhino

input_data = (2100,4.5,50,6.5,1100,50,9.5,7.5,8.5,1400,4.5,30,10,1900,50,9.5,7,9)
animalvs(input_data)

#Training using KNeighbour CLassifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
y_pred_knn = knn.predict(x_test)
print(f"KNN Accuracy: {accuracy_score(y_test, y_pred_knn)}")

#Making predictive system for KNN model

In [ ]:
def animal(input_data):


    # changing the input data to numpy array

    input_data_as_numpy = np.asarray(input_data)

    # reshape the array as we are predicting for one instance

    input_data_reshaped = input_data_as_numpy.reshape(1,-1)

    # standardize the array

    std_data = scaler.transform(input_data_reshaped)
    prediction = model.predict(std_data)
    print(prediction[0])

    if (prediction[0][0]==0.5):
      print(" it is a tie")
    elif(prediction[0][0]>0.5):
      print(f" there is  a {prediction[0][0]*100:.2f} % chance that the first animal will win ")
    else:
      print(f" there is  a {prediction[0][1] *100:.2f} % chance that the second animal will win ")




In [ ]:
# rhino vs hippo : the winner is rhino

input_data = (2100,4.5,50,6.5,1100,50,9.5,7.5,8.5,1400,4.5,30,10,1900,50,9.5,7,9)
animal(input_data)

In [ ]:
# african lion vs bengal tiger : the winner is bengal tiger

input_data =(190,6.5,58,9,650,12,6,4.5,7.5,230,6,65,8.5,1050,13,7,4.5,8)
animal(input_data)

In [ ]:
# walrus vs nile crocodile : the winner is nile crocodile

input_data = (1200, 4.5, 35, 6.5, 200, 40, 8, 9, 7,	500, 4.5, 35, 9.5, 4400, 50, 9, 7.5, 8)
animal(input_data)